In [1]:
# KNOWN ISSUES:
# If ACM has document open, script will stall
# Documents from SharePoint open read-only: https://excel.uservoice.com/forums/304921-excel-for-windows-desktop-application/suggestions/10205019-read-only-mode?page=1&per_page=20
#     - fixed by using "wb.save(file_path)" instead of "wb.save()"

In [2]:
from cyschoolhousesuite import *
import xlwings as xw # Excel must be closed prior to running script

CP_df = get_report("00O1a000002rm2S")
CP_df = CP_df.loc[CP_df['Program: Program Name'].str.contains("Attendance")].copy()
CP_df.sort_values(by=['Account Name', 'Student: Student Name'], inplace=True)
CP_df = CP_df[[
    'Account Name',
    'Student: Student Name',
    'Student Id',
    'Student: Grade',
    'Intervention Primary Staff: Staff Name'
]]

In [3]:
sch_ref_path = r'Z:\ChiPrivate\Chicago Data and Evaluation\SY18\SY18 Team Placement\SY18 FINAL Team Placements.xlsx'
sch_ref_df = pd.read_excel(sch_ref_path, sheet_name="School Data")
sch_ref_df = sch_ref_df.loc[~sch_ref_df['School'].isnull()]

In [4]:
# Update existing files
for index, row in sch_ref_df.iterrows():
    try:
        file_path = ("Z:\\" + row['cyconnect Folder'] + "\\" + "SY18 Attendance Tracker - "
                     + row['Informal School Name'] + ".xlsx")
        wb = xw.Book(file_path)
#         wb.app.display_alerts = False
        sht = wb.sheets["Enter Focus List Here"]
        df = CP_df.loc[CP_df['Account Name']==row['School']].copy()
        df.drop_duplicates(subset='Student Id', inplace=True)
        sht.range('A3:E300').value = "."
        sht.range('A3').options(index=False, header=False).value = df.copy()
        wb.save(file_path)
    except Exception as e:
        print(f"Failed: {row['School']}: {e}")
        continue
    wb.close()

In [ ]:
## for future reference this might be a better way to close windows instead of wb.close()
# app = xw.apps.active
# app.quit()